# What is Linear Regression
In statistics, linear regression is a linear approach to modeling the relationship between a scalar response and one or more explanatory variables. The case of one explanatory variable is called simple linear regression. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
Report=pd.read_excel('C://Users//eorsswv//Reporting//Data//Automation//4G.xlsx')
Report=Report.drop(['Date'], axis=1)
Report.head()


,Equivalent Downlink Average Throughput BH Value,Equivalent Uplink Average Throughput BH Value,Traffic Volume DL Cell PDCP [MB] BH Value,Traffic Volume UL Cell PDCP [MB] BH Value,Maximum RRC-Connected User Nr_PA BH Value,Mean RRC-Connected User Nr BH Value,Initial E-RAB Setup SR BH Value,Average E-RAB Nr per User BH Value,Added E-RAB Setup SR BH Value,ADMITFAIL_CELLBLOCK BH Value,...,Nr of Initial QCI6 E-RABs Attempted to Setup BH Value,Nr of Initial QCI5 E-RABs Attempted to Setup BH Value,Nr of Initial QCI4 E-RABs Attempted to Setup BH Value,Nr of Initial QCI3 E-RABs Attempted to Setup BH Value,Nr of initial QCI2 E-RABs Attempted to Setup BH Value,Nr of initial QCI1 E-RABs Attempted to Setup BH Value,Nr of initial E-RABs Failed to Setup due to Receiving RRC Re-establishment Request BH Value,Nr of initial E-RABs Failed to Setup due to Other Reason BH Value,Nr Of Abnormal Release Active E-RAB BH Value,Active E-RAB Abnormal Release based on successfully established E-RAB BH Value
0,41.86 Mbps,23.19 Mbps,23168815.68 MB,2257987.84 MB,436875,219763.14,99.90,5.16,99.66,0,...,214832,2095844,0,0,0,78,0,4170,11586,11586
1,42.03 Mbps,23.48 Mbps,23650071.00 MB,2133090.04 MB,409148,202610.47,99.92,5.24,99.67,0,...,193510,2023957,0,0,0,53,0,1881,7884,7884
2,42.20 Mbps,23.49 Mbps,23053338.56 MB,1984684.12 MB,444172,220973.19,99.90,5.15,99.70,0,...,229213,2081219,0,0,0,201,0,1705,10773,10773
3,42.36 Mbps,23.47 Mbps,22648952.88 MB,1889506.71 MB,432565,217649.91,99.91,5.21,99.71,0,...,230449,2010601,0,0,0,45,0,1696,10320,10320
4,42.49 Mbps,23.42 Mbps,22061532.23 MB,1882065.31 MB,428833,215099.30,99.90,5.47,99.70,0,...,251529,2009511,0,0,0,53,5,1646,10275,10275


In [2]:
evaluation = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

In [3]:
# Removing unwanted strings in dataset
Report['Equivalent Downlink Average Throughput BH Value'] = Report['Equivalent Downlink Average Throughput BH Value'].map(lambda x: x.rstrip('Mbps'))
Report['Equivalent Uplink Average Throughput BH Value'] = Report['Equivalent Uplink Average Throughput BH Value'].map(lambda x: x.rstrip('Mbps'))
Report['Traffic Volume DL Cell PDCP [MB] BH Value'] = Report['Traffic Volume DL Cell PDCP [MB] BH Value'].map(lambda x: x.rstrip('MB'))
Report['Traffic Volume UL Cell PDCP [MB] BH Value'] = Report['Traffic Volume UL Cell PDCP [MB] BH Value'].map(lambda x: x.rstrip('MB'))
Report['Maximum Cell UL PDCP Throughput BH Value'] = Report['Maximum Cell UL PDCP Throughput BH Value'].map(lambda x: x.rstrip('Mbps'))
Report['Maximum Cell DL PDCP Throughput BH Value'] = Report['Maximum Cell DL PDCP Throughput BH Value'].map(lambda x: x.rstrip('Mbps'))
Report['Average Cell DL PDCP Throughput [Mbps] BH Value'] = Report['Average Cell DL PDCP Throughput [Mbps] BH Value'].map(lambda x: x.rstrip('Mbps'))
Report['Average Cell UL PDCP Throughput [Mbps] BH Value'] = Report['Average Cell UL PDCP Throughput [Mbps] BH Value'].map(lambda x: x.rstrip('Mbps'))
Report=Report.astype(float)

In [4]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

In [5]:
Report3=pd.read_excel('C://Users//eorsswv//Reporting//Data//Automation//4G.xlsx','Sheet2')
Report3['Equivalent Downlink Average Throughput BH Value'] = Report3['Equivalent Downlink Average Throughput BH Value'].map(lambda x: x.rstrip('Mbps'))
Report3['Traffic Volume DL Cell PDCP [MB] BH Value'] = Report3['Traffic Volume DL Cell PDCP [MB] BH Value'].map(lambda x: x.rstrip('MB'))
Report3.describe()
Report4=Report3.iloc[:,1:8]  ##Removed Date
Report4.head()

Report4['Equivalent Downlink Average Throughput BH Value']=Report4['Equivalent Downlink Average Throughput BH Value'].astype(float)
Report4['Traffic Volume DL Cell PDCP [MB] BH Value']=Report4['Traffic Volume DL Cell PDCP [MB] BH Value'].astype(float)
Report4['Maximum RRC-Connected User Nr_PA BH Value']=Report4['Maximum RRC-Connected User Nr_PA BH Value'].astype(float)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(Report4)

In [7]:
X = Report4[['DL 4G Avg IP Scheduled User Throughput QCI8 BH Value','Traffic Volume DL Cell PDCP [MB] BH Value','Maximum RRC-Connected User Nr_PA BH Value','Mean RRC-Connected User Nr BH Value','PDSCH PRB Utilization Rate BH Value','DL PRB Utilization Rate BH Value']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
Y = Report4['Equivalent Downlink Average Throughput BH Value']


# Apply Linear Regression


In [8]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

# First we will apply Linear Regression

In [9]:

train_data_dm,test_data_dm = train_test_split(Report4,train_size = 0.8,random_state=3)
regr = linear_model.LinearRegression()
features = ['DL 4G Avg IP Scheduled User Throughput QCI8 BH Value','Traffic Volume DL Cell PDCP [MB] BH Value','Maximum RRC-Connected User Nr_PA BH Value','Mean RRC-Connected User Nr BH Value','PDSCH PRB Utilization Rate BH Value','DL PRB Utilization Rate BH Value']
complex_model_1 = linear_model.LinearRegression()
complex_model_1.fit(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value'])

print('Intercept: {}'.format(complex_model_1.intercept_))
print('Coefficients: {}'.format(complex_model_1.coef_))

pred = complex_model_1.predict(test_data_dm[features])
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['Equivalent Downlink Average Throughput BH Value'],pred)),'.3f'))
rtrcm = float(format(complex_model_1.score(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_1.score(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_1.score(test_data_dm[features],test_data_dm['Equivalent Downlink Average Throughput BH Value']),'.3f'))
artecm = float(format(adjustedR2(complex_model_1.score(test_data_dm[features],test_data_dm['Equivalent Downlink Average Throughput BH Value']),test_data_dm.shape[0],len(features)),'.3f'))
cv = float(format(cross_val_score(complex_model_1,Report4[features],Report4['Equivalent Downlink Average Throughput BH Value'],cv=5).mean(),'.3f'))


r = evaluation.shape[0]
evaluation.loc[r] = ['Multiple Regression-1','selected features',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)

Intercept: 45.988503616682316
Coefficients: [ 2.72996710e-02  3.29987451e-07  1.33150764e-05 -2.90995939e-05
 -5.33771383e-01 -4.21865952e-01]


C:\Users\eorsswv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
0,Multiple Regression-1,selected features,0.21,0.874,0.864,0.893,0.848,0.1


# Secondly we will apply Ridge Regression

In [10]:


complex_model_2 = linear_model.Ridge(alpha=1)
complex_model_2.fit(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value'])

print('Intercept: {}'.format(complex_model_2.intercept_))
print('Coefficients: {}'.format(complex_model_2.coef_))

pred = complex_model_2.predict(test_data_dm[features])
rmsecm= float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['Equivalent Downlink Average Throughput BH Value'],pred)),'.3f'))
rtrcm= float(format(complex_model_2.score(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value']),'.3f'))
artrcm= float(format(adjustedR2(complex_model_2.score(train_data_dm[features],train_data_dm['Equivalent Downlink Average Throughput BH Value']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_2.score(test_data_dm[features],test_data_dm['Equivalent Downlink Average Throughput BH Value']),'.3f'))
artecm = float(format(adjustedR2(complex_model_2.score(test_data_dm[features],test_data_dm['Equivalent Downlink Average Throughput BH Value']),test_data_dm.shape[0],len(features)),'.3f'))
cv = float(format(cross_val_score(complex_model_2,Report4[features],Report4['Equivalent Downlink Average Throughput BH Value'],cv=5).mean(),'.3f'))

r = evaluation.shape[0]

evaluation.loc[r] = ['Ridge Regression for Alpha=1','selected features',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)

Intercept: 45.20342240528606
Coefficients: [ 4.05040010e-02  2.70244558e-07  1.37434595e-05 -3.44938483e-05
 -3.28679499e-01 -3.84800202e-01]


,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
1,Ridge Regression for Alpha=1,selected features,0.232,0.870,0.860,0.870,0.815,0.378
0,Multiple Regression-1,selected features,0.210,0.874,0.864,0.893,0.848,0.100


# 